<a href="https://colab.research.google.com/github/CLERC-Tom/Train_A_Lora/blob/main/Training_LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash

if [ ! -d /content/diffusers ]; then
  git clone https://github.com/huggingface/diffusers.git /content/diffusers
fi

pip install --quiet \
     git+https://github.com/huggingface/diffusers.git \
     accelerate \
     transformers \
     safetensors \
     bitsandbytes

cd /content/diffusers/examples/dreambooth


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(token="YOUR_TOKEN")  # put your hugging face token


In [ ]:
!nvidia-smi


Tue Apr 22 20:29:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             42W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%bash

cd /content/diffusers/examples/dreambooth

nvidia-smi

accelerate launch \
  --mixed_precision="fp16" \
  --num_processes=1 \
  train_dreambooth.py \
    --pretrained_model_name_or_path "runwayml/stable-diffusion-v1-5" \
    --instance_data_dir      "YOUR_PATH_OF_TRAINING_IMAGES" \  # Put your folder with your images
    --instance_prompt        "CUSTOM" \ # Put a custom name for your result
    --resolution             128 \
    --train_batch_size       1 \
    --gradient_accumulation_steps 1 \
    --learning_rate          5e-6 \
    --lr_scheduler           constant \
    --lr_warmup_steps        0 \
    --max_train_steps        400 \
    --dataloader_num_workers 2 \
    --gradient_checkpointing \
    --use_8bit_adam \
    --enable_xformers_memory_efficient_attention \
    --output_dir             "/content/dreambooth_output" \
    --push_to_hub


Tue Apr 22 20:20:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-04-22 20:20:39.765398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745353239.784687    8008 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745353239.790762    8008 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 20:20:39.809962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instruction

In [ ]:
!find /content/dreambooth_output -type f -name "*.safetensors"


/content/dreambooth_output/vae/diffusion_pytorch_model.safetensors
/content/dreambooth_output/text_encoder/model.safetensors
/content/dreambooth_output/safety_checker/model.safetensors
/content/dreambooth_output/unet/diffusion_pytorch_model.safetensors


In [ ]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive')

src = "/content/dreambooth_output/unet/diffusion_pytorch_model.safetensors"

dst_folder = "/content/drive/MyDrive/dreambooth_output"
os.makedirs(dst_folder, exist_ok=True)

dst = os.path.join(dst_folder, "unet_finetuned.safetensors")
shutil.copy(src, dst)

print("✅ Copié :", dst)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Copié : /content/drive/MyDrive/dreambooth_output/unet_finetuned.safetensors


In [ ]:
!zip -r /content/drive/MyDrive/dreambooth_output.zip /content/dreambooth_output



  adding: content/dreambooth_output/ (stored 0%)
  adding: content/dreambooth_output/logs/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/1745353247.996663/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/1745353247.996663/events.out.tfevents.1745353247.1f9a25c54558.8008.1 (deflated 56%)
  adding: content/dreambooth_output/logs/dreambooth/1745353247.9984024/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/1745353247.9984024/hparams.yml (deflated 52%)
  adding: content/dreambooth_output/logs/dreambooth/1745352767.1167977/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/1745352767.1167977/hparams.yml (deflated 53%)
  adding: content/dreambooth_output/logs/dreambooth/1745352116.6654434/ (stored 0%)
  adding: content/dreambooth_output/logs/dreambooth/1745352116.6654434/hparams.yml (deflated 53%)
  adding: content/dreambooth_output/logs/dreambooth/events.out.tfev